In [5]:
import sqlite3
import pandas as pd

# Database file (auto-created in project folder)
DB_FILE = "face_recognition.db"


def create_database():
    """Create SQLite database and persons table"""
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS persons (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL UNIQUE,
            age INTEGER,
            location TEXT
        )
    ''')
    conn.commit()
    conn.close()


def insert_person(name, age, location):
    """Insert a new person"""
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    try:
        cursor.execute("INSERT INTO persons (name, age, location) VALUES (?, ?, ?)", (name, age, location))
        conn.commit()
        print(f"Inserted: {name}, Age {age}, Location {location}")
    except sqlite3.IntegrityError:
        print(f"Person '{name}' already exists.")
    conn.close()


def fetch_all_persons():
    """Return all persons as Pandas DataFrame"""
    conn = sqlite3.connect(DB_FILE)
    df = pd.read_sql_query("SELECT * FROM persons", conn)
    conn.close()
    return df


if __name__ == "__main__":
    create_database()

    insert_person("ronaldo", 40, "Funchal, Portugal")
    insert_person("sinner", 24, "San Candido, Italy")
    insert_person("cummins", 32, "Westmead, Australia")
    insert_person("shelton", 22, "Georgia, United States")

    print(fetch_all_persons())

Inserted: ronaldo, Age 40, Location Funchal, Portugal
Inserted: sinner, Age 24, Location San Candido, Italy
Inserted: cummins, Age 32, Location Westmead, Australia
Inserted: shelton, Age 22, Location Georgia, United States
   id     name  age                location
0   1  ronaldo   40       Funchal, Portugal
1   2   sinner   24      San Candido, Italy
2   3  cummins   32     Westmead, Australia
3   4  shelton   22  Georgia, United States


In [ ]:
import cv2
import dlib
import numpy as np
import os
import sqlite3


predictor_path = r"shape_predictor_68_face_landmarks.dat"
face_rec_model_path = r"dlib_face_recognition_resnet_model_v1.dat"

# Load models
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)
face_rec_model = dlib.face_recognition_model_v1(face_rec_model_path)


DB_FILE = "face_recognition.db"

def fetch_person_info(name):
    """Fetch person details by name"""
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM persons WHERE name = ?", (name,))
    row = cursor.fetchone()
    conn.close()
    if row:
        return {"id": row[0], "name": row[1], "age": row[2], "location": row[3]}
    return None


reference_folder = "images"
reference_encodings, reference_names = [], []

for file_name in os.listdir(reference_folder):
    if file_name.endswith((".jpg", ".png", ".jpeg")):
        img_path = os.path.join(reference_folder, file_name)
        img = cv2.imread(img_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        faces = detector(img_rgb)
        if len(faces) == 0:
            continue
        shape = predictor(img_rgb, faces[0])
        encoding = np.array(face_rec_model.compute_face_descriptor(img_rgb, shape))
        reference_encodings.append(encoding)
        reference_names.append(os.path.splitext(file_name)[0])

print(f"Loaded {len(reference_encodings)} reference faces.")


image_path = "pat_i.jpg"
img = cv2.imread(image_path)
if img is None:
    print("Error: Could not load image.")
    exit()

img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
faces = detector(img_rgb)

if len(faces) == 0:
    print("No faces detected in the image.")
else:
    for face in faces:
        x1, y1, x2, y2 = face.left(), face.top(), face.right(), face.bottom()
        shape = predictor(img_rgb, face)
        encoding = np.array(face_rec_model.compute_face_descriptor(img_rgb, shape))

        distances = np.linalg.norm(reference_encodings - encoding, axis=1)
        min_dist_index = np.argmin(distances)

        if distances[min_dist_index] < 0.6:
            person_name = reference_names[min_dist_index]
            info = fetch_person_info(person_name)
            if info:
                label = f"{info['name']} | Age: {info['age']} | {info['location']}"
            else:
                label = person_name
            color = (0, 255, 0)
        else:
            label = "Unknown"
            color = (0, 0, 255)

        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
        cv2.putText(img, label, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)


cv2.imshow("Face Recognition - Single Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

Loaded 4 reference faces.


In [ ]:
import cv2
import dlib
import numpy as np
import os
import sqlite3

# Paths to dlib models
predictor_path = r"shape_predictor_68_face_landmarks.dat"
face_rec_model_path = r"dlib_face_recognition_resnet_model_v1.dat"

# Load models
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)
face_rec_model = dlib.face_recognition_model_v1(face_rec_model_path)

# Database file
DB_FILE = "face_recognition.db"

def fetch_person_info(name):
    """Fetch person details by name"""
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM persons WHERE name = ?", (name,))
    row = cursor.fetchone()
    conn.close()
    if row:
        return {"id": row[0], "name": row[1], "age": row[2], "location": row[3]}
    return None


# Load reference images
reference_folder = "images"
reference_encodings, reference_names = [], []

for file_name in os.listdir(reference_folder):
    if file_name.endswith((".jpg", ".png", ".jpeg")):
        img_path = os.path.join(reference_folder, file_name)
        img = cv2.imread(img_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        faces = detector(img_rgb)
        if len(faces) == 0:
            continue
        shape = predictor(img_rgb, faces[0])
        encoding = np.array(face_rec_model.compute_face_descriptor(img_rgb, shape))
        reference_encodings.append(encoding)
        reference_names.append(os.path.splitext(file_name)[0])

print(f"Loaded {len(reference_encodings)} reference faces.")


# Video File
video_path = "vid/s.mp4"
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Could not open video file.")
    exit()


# Video
while True:
    ret, frame = cap.read()
    if not ret:
        print("End of video reached.")
        break

    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    faces = detector(img_rgb)

    for face in faces:
        x1, y1, x2, y2 = face.left(), face.top(), face.right(), face.bottom()
        shape = predictor(img_rgb, face)
        encoding = np.array(face_rec_model.compute_face_descriptor(img_rgb, shape))

        distances = np.linalg.norm(reference_encodings - encoding, axis=1)
        min_dist_index = np.argmin(distances)

        if distances[min_dist_index] < 0.6:
            person_name = reference_names[min_dist_index]
            info = fetch_person_info(person_name)
            if info:
                label = f"{info['name']} | Age: {info['age']} | {info['location']}"
            else:
                label = person_name
            color = (0, 255, 0)
        else:
            label = "Unknown"
            color = (0, 0, 255)

        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, label, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    cv2.imshow("Face Recognition Video", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [1]:
import cv2
import dlib
import numpy as np
import os
import sqlite3

# Paths to dlib models
predictor_path = r"shape_predictor_68_face_landmarks.dat"
face_rec_model_path = r"dlib_face_recognition_resnet_model_v1.dat"

# Load models
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)
face_rec_model = dlib.face_recognition_model_v1(face_rec_model_path)

# Database file
DB_FILE = "face_recognition.db"

def fetch_person_info(name):
    """Fetch person details by name"""
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM persons WHERE name = ?", (name,))
    row = cursor.fetchone()
    conn.close()
    if row:
        return {"id": row[0], "name": row[1], "age": row[2], "location": row[3]}
    return None


# Load reference images
reference_folder = "images"
reference_encodings, reference_names = [], []

for file_name in os.listdir(reference_folder):
    if file_name.endswith((".jpg", ".png", ".jpeg")):
        img_path = os.path.join(reference_folder, file_name)
        img = cv2.imread(img_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        faces = detector(img_rgb)
        if len(faces) == 0:
            continue
        shape = predictor(img_rgb, faces[0])
        encoding = np.array(face_rec_model.compute_face_descriptor(img_rgb, shape))
        reference_encodings.append(encoding)
        reference_names.append(os.path.splitext(file_name)[0])

print(f"Loaded {len(reference_encodings)} reference faces.")


# Webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    faces = detector(img_rgb)

    for face in faces:
        x1, y1, x2, y2 = face.left(), face.top(), face.right(), face.bottom()
        shape = predictor(img_rgb, face)
        encoding = np.array(face_rec_model.compute_face_descriptor(img_rgb, shape))

        distances = np.linalg.norm(reference_encodings - encoding, axis=1)
        min_dist_index = np.argmin(distances)

        if distances[min_dist_index] < 0.6:
            person_name = reference_names[min_dist_index]
            info = fetch_person_info(person_name)
            if info:
                label = f"{info['name']} | Age: {info['age']} | {info['location']}"
            else:
                label = person_name
            color = (0, 255, 0)
        else:
            label = "Unknown"
            color = (0, 0, 255)

        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, label, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    cv2.imshow("Face Recognition System", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Loaded 4 reference faces.
